In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
import os
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras import backend as K
from keras.utils import np_utils
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
%matplotlib inline
import matplotlib.image as mpimg
import splitfolders


In [2]:
# splitfolders.ratio(input="/car_plates/", output="output", seed=1337, ratio=(.8, 0.1,0.1))

dataset=pd.read_csv(r'/car_plates/PoolLabels.csv')
dataset.head()

,img.name,label
0,car001.jpg,ر ب ل ١ ٨ ٢ ٧
1,car002.jpg,ى ن ر ٤ ٦ ٢ ٥
2,car003.jpg,ع س ط ١ ٣ ٧ ٥
3,car004.jpg,أ ج س ٤ ١ ٤ ٧
4,car005.jpg,ق ج ل ٢ ٧ ٣ ٩


In [3]:
training_dir='/car_plates/output/train/Pics'
test_dir='/car_plates/output/test/Pics'
validation_dir='/car_plates/output/val/Pics'
images= dataset['img.name']
labels=dataset['label']
images.head()

0    car001.jpg
1    car002.jpg
2    car003.jpg
3    car004.jpg
4    car005.jpg
Name: img.name, dtype: object

In [4]:
# from PIL import Image
# directory = '/car_plates/Pics/'

# training_img=[]
# y=[]

# for i in range(images.__len__()):
#     images[i]=directory+images[i]
    


# for i in range(training_img.__len__()):
#     x.append(training_img[i])
#     y.append(training_label[i])



In [5]:
# plt.title(labels[200])
# # plt.figure(figsize = (12,10))
# # plt.subplot(1,2,1)
# # plt.title('Original image')
# img = mpimg.imread(images[200])
# print(img)
# plt.imshow(img)


In [6]:


# x_train,x_test,y_train,y_test=train_test_split(training_img,training_label,train_size=0.7)


In [7]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
    rescale = 1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
   
    fill_mode='nearest'
    )
test_datagen = ImageDataGenerator(rescale = 1./255)

In [8]:
dataset['img.name'].head()

0    car001.jpg
1    car002.jpg
2    car003.jpg
3    car004.jpg
4    car005.jpg
Name: img.name, dtype: object

In [9]:
train_generator = train_datagen.flow_from_dataframe(
    dataframe=dataset,
    directory=training_dir,
                  x_col='img.name',
                  y_col='label',
                #   target_size =(300, 300),  # target_size = input image size
                  
                  class_mode ='raw')

# test_generator = test_datagen.flow_from_dataframe(
#                     x_test,
#                     target_size =(150, 150),
#                     batch_size = 20,
#                     class_mode ='binary')

Found 2310 validated image filenames.


c:\anaconda3\envs\tf2.4\lib\site-packages\keras\preprocessing\image.py:1137: UserWarning: Found 587 invalid image filename(s) in x_col="img.name". These filename(s) will be ignored.
  warnings.warn(


In [10]:
train_generator.batch_size
train_generator.samples
train_generator.dtype


'float32'

In [11]:
train_generator.image_shape
dataset.head()

,img.name,label
0,car001.jpg,ر ب ل ١ ٨ ٢ ٧
1,car002.jpg,ى ن ر ٤ ٦ ٢ ٥
2,car003.jpg,ع س ط ١ ٣ ٧ ٥
3,car004.jpg,أ ج س ٤ ١ ٤ ٧
4,car005.jpg,ق ج ل ٢ ٧ ٣ ٩


In [26]:
from keras.layers import Activation, Dense

model =tf.keras.models.Sequential([
 tf.keras.layers.Conv2D(32, (3,3), activation='relu' ,
 input_shape=(300, 300, 3),padding='same'),
 tf.keras.layers.MaxPooling2D(2, 2),
 tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
 tf.keras.layers.MaxPooling2D(2,2),
 tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
 tf.keras.layers.MaxPooling2D(2,2),
 tf.keras.layers.Flatten(),
 tf.keras.layers.Dense(512, activation='relu'),
 tf.keras.layers.Dense(512, activation='relu'),
 tf.keras.layers.Dense(7, activation='softmax')  
 
    
 ])




In [27]:
model.compile(loss='binary_crossentropy',
 optimizer='adam',
# RMSprop(lr=0.001)
 metrics=['accuracy'])

In [28]:
characters = set(char for label in labels for char in label)
print(characters)

{' ', '4', '٣', '١', 'ك', 'ز', '۳', '٦', 'د', '\u200d', '۷', '8', '۹', '9', '٥', '٠', '6', 'ه', '1', '۸', '٧', '2', '7', '٩', 'ط', 'ج', 'ا', 'ل', 'س', 'ق', 'ف', '٤', '٢', 'ص', '\xa0', 'م', 'ي', 'ح', 'ى', 'ر', 'ع', 'و', '٨', '5', 'ب', '۱', '3', 'أ', 'ن', '۲'}


In [29]:
from keras.layers import StringLookup

arabic_num_alph=[
    "١",
            "٢",
            "٣",
            "٤",
            "٥",
            "٦",
            "٧",
            "٨",
            "٩",
            "٠",
            "أ",
            'ص', 'ف','ل','ح','س', 'د', 'ي', 'ز','ن','ك',"ب",'ط', 'ا','ج', 'ر', 'ى', 'م', 'ق','ع', 'ه']



char_to_num = tf.keras.layers.StringLookup(
    vocabulary=list(arabic_num_alph), mask_token=None
)

# Mapping integers back to original characters
num_to_char = tf.keras.layers.StringLookup(
    vocabulary=char_to_num.get_vocabulary(), mask_token=None, invert=True
)

print(num_to_char)
print(arabic_num_alph)

['١', '٢', '٣', '٤', '٥', '٦', '٧', '٨', '٩', '٠', 'أ', 'ص', 'ف', 'ل', 'ح', 'س', 'د', 'ي', 'ز', 'ن', 'ك', 'ب', 'ط', 'ا', 'ج', 'ر', 'ى', 'م', 'ق', 'ع', 'ه']


In [30]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_6 (Conv2D)           (None, 300, 300, 32)      896       
                                                                 
 max_pooling2d_6 (MaxPooling  (None, 150, 150, 32)     0         
 2D)                                                             
                                                                 
 conv2d_7 (Conv2D)           (None, 148, 148, 64)      18496     
                                                                 
 max_pooling2d_7 (MaxPooling  (None, 74, 74, 64)       0         
 2D)                                                             
                                                                 
 conv2d_8 (Conv2D)           (None, 72, 72, 128)       73856     
                                                                 
 max_pooling2d_8 (MaxPooling  (None, 36, 36, 128)     

In [31]:
model.fit(
    train_generator,
    
    epochs=20,
    )

Epoch 1/20


ValueError: in user code:

    File "c:\anaconda3\envs\tf2.4\lib\site-packages\keras\engine\training.py", line 1249, in train_function  *
        return step_function(self, iterator)
    File "c:\anaconda3\envs\tf2.4\lib\site-packages\keras\engine\training.py", line 1233, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "c:\anaconda3\envs\tf2.4\lib\site-packages\keras\engine\training.py", line 1222, in run_step  **
        outputs = model.train_step(data)
    File "c:\anaconda3\envs\tf2.4\lib\site-packages\keras\engine\training.py", line 1024, in train_step
        loss = self.compute_loss(x, y, y_pred, sample_weight)
    File "c:\anaconda3\envs\tf2.4\lib\site-packages\keras\engine\training.py", line 1082, in compute_loss
        return self.compiled_loss(
    File "c:\anaconda3\envs\tf2.4\lib\site-packages\keras\engine\compile_utils.py", line 265, in __call__
        loss_value = loss_obj(y_t, y_p, sample_weight=sw)
    File "c:\ana